In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import os

In [ ]:
import skimage
from skimage.transform import resize

In [ ]:
BS = 8
imageSize = 640
target_dims = (imageSize, imageSize, 3)
num_classes = 29

train_len = 50
train_dir = r'C:/Users/vatsa/OneDrive/Desktop/Covid-19-X-Ray-Recognition/dataset/'

def get_data(folder):
    """
    Load the data and labels from the given folder.
    """
    X = np.empty((train_len, imageSize, imageSize, 3), dtype=np.float32)
    y = np.empty((train_len,), dtype=np.int)
    cnt = 0

    for folderName in os.listdir(folder):
        if not folderName.startswith('.'):
            if folderName in ['normal']:
                label = 0
            elif folderName in ['covid']:
                label = 1
                
            for image_filename in os.listdir(folder + folderName):
                img_file = cv2.imread(folder + folderName + '/' + image_filename)
                if img_file is not None:
                    img_file = skimage.transform.resize(img_file, (imageSize, imageSize, 3))
                    img_arr = np.asarray(img_file).reshape((-1, imageSize, imageSize, 3))
                    
                    X[cnt] = img_arr
                    y[cnt] = label
                    cnt += 1
#                     X.append(img_arr)
#                     y.append(label)
#     X = np.asarray(X)
#     y = np.asarray(y)
    return X,y
X_data, y_data = get_data(train_dir)

In [ ]:
X_data.shape

In [ ]:
y_data.shape

In [ ]:
from keras.utils import to_categorical

In [ ]:
Y_data = to_categorical(y_data)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.3,
                                                    stratify=Y_data, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

### Creating The Model

In [ ]:
from keras.layers import Convolution2D, Dense, Dropout, Flatten, Activation, MaxPooling2D
from keras.models import Sequential

In [ ]:
model = Sequential()

model.add(Convolution2D(64, (5, 5), input_shape=(640, 640, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Convolution2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Convolution2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(2, activation='sigmoid'))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
trainAug = ImageDataGenerator(rotation_range=15,
                              fill_mode="nearest")

In [ ]:
testAug = ImageDataGenerator(rotation_range=10,
                              fill_mode="nearest")

In [ ]:
model.fit_generator(trainAug.flow(X_train, y_train, batch_size=BS, shuffle=True),
                    steps_per_epoch=len(X_train),
                    validation_data=testAug.flow(X_test, y_test, batch_size=BS, shuffle=True),
                    validation_steps=len(X_test),
                    epochs=25,
                    verbose=2)
